In [58]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
pd.set_option('display.max_columns',None)

In [3]:
import os
import pandas as pd

# Caminho para a pasta contendo os arquivos CSV
pasta = r"C:\Users\natal\OneDrive\Documentos\Pessoal\jump\Calvin-20240319T191946Z-001\Calvin"

# Lista para armazenar os DataFrames de cada arquivo CSV
dataframes = []

# Iterar sobre os arquivos na pasta
for arquivo in os.listdir(pasta):
    if arquivo.endswith('.csv'):
        caminho_arquivo = os.path.join(pasta, arquivo)
        df = pd.read_csv(caminho_arquivo,encoding = 'latin-1',sep=';')
        df['nome_arquivo'] = arquivo
        dataframes.append(df)

# Concatenar todos os DataFrames em um único DataFrame
df_final = pd.concat(dataframes, ignore_index=True)



In [4]:
df_final.tipo.unique()

array(['FATURAMENTO', 'PAGAMENTOS', 'DEVOLUCAO'], dtype=object)

In [5]:
df_final.head()

,Data corte 13/07/2022,cliente,cidade,uf,tipo,documento,forma_pgto,valor,imposto,encargo,comissao_base,comissao,adiantamento,data_faturamento,data_lancamento,data_pagamento,parcela,representante,indicador_fiscal_terceiro,descricao_fiscal_terceiro,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Dados,Unnamed: 29,Unnamed: 30,Unnamed: 31,nome_arquivo,Unnamed: 28,Unnamed: 32
0,DEPOIS,AAC COMERCIO DE CONFECCOE,FLORIANOPOLIS,SC,FATURAMENTO,940997,28/49/70/91/112 FRANQUIA,"6230,04",0,"124,6","6105,44","305,27","152,64",04/04/2023,NaN,NaN,NaN,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,NaN,NaN,NaN,NaN,NaN,tipo,Data corte 13/07/2022,cliente,Soma de comissao_base,Soma de comissao,Soma de adiantamento,50% Desconto Acordo Cml,abril 23.csv,NaN,NaN
1,DEPOIS,AAC COMERCIO DE CONFECCOE,FLORIANOPOLIS,SC,PAGAMENTOS,928624,28/49/70/91/112 FRANQUIA,"1098,08",0,"21,96","1076,12","53,81","26,9",10/02/2023,10/02/2023,06/04/2023,B,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,NaN,NaN,NaN,NaN,NaN,PAGAMENTOS,DEPOIS,DOUBLE CRICIUMA,"63.700,59","3.184,98","1.592,53","1.592,49",abril 23.csv,NaN,NaN
2,DEPOIS,AAC COMERCIO DE CONFECCOE,FLORIANOPOLIS,SC,PAGAMENTOS,928624,28/49/70/91/112 FRANQUIA,"688,99",0,"13,78","675,21","33,76","16,88",10/02/2023,10/02/2023,26/04/2023,C,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,NaN,NaN,NaN,NaN,NaN,Total Geral,NaN,NaN,"63.700,59","3.184,98","1.592,53",NaN,abril 23.csv,NaN,NaN
3,DEPOIS,AAC COMERCIO DE CONFECCOE,FLORIANOPOLIS,SC,PAGAMENTOS,936605,28/49/70/91/112 FRANQUIA,"1284,36",0,"25,69","1258,67","62,93","31,47",15/03/2023,15/03/2023,12/04/2023,A,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,abril 23.csv,NaN,NaN
4,DEPOIS,ADRIANA MARIANI,ALFREDO WAGNER,SC,FATURAMENTO,940891,28/49/70/91/112 (JUNIOR),7244,0,"144,88","7099,12","354,96","177,48",04/04/2023,NaN,NaN,NaN,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,abril 23.csv,NaN,NaN


O arquivo de comissão da Calvin Klein vem com 3 tipos de operações (FATURAMENTO', 'PAGAMENTOS', 'DEVOLUCAO').
Quando é do tipo FATURAMENTO, nota-se que temos apenas o pedido e nenhum pagamento envolvido. Vamos utilizar este tipo para fazer a provisão da receita (TABELA PROVISÃO).
Quando o tipo é PAGAMENTOS ou DEVOLUÇÃO, temos a receita de fato obtida. Vamos utilizar para criar a TABELA RECEITA REAL


In [6]:
calvin = df_final[['nome_arquivo','cliente', 'cidade', 'uf', 'tipo',
       'documento', 'forma_pgto', 'valor', 'imposto', 'encargo',
       'comissao_base', 'comissao', 'adiantamento', 'data_faturamento',
       'data_lancamento', 'data_pagamento', 'parcela', 'representante',
       'indicador_fiscal_terceiro', 'descricao_fiscal_terceiro']]

In [7]:
calvin.valor = calvin.valor.str.replace(',', '.').astype(float)
calvin.encargo = calvin.encargo.str.replace(',', '.').astype(float)
calvin.comissao_base = calvin.comissao_base.str.replace(',', '.').astype(float)
calvin.comissao = calvin.comissao.str.replace(',', '.').astype(float)
calvin.adiantamento = calvin.adiantamento.str.replace(',', '.').astype(float)

In [8]:
from datetime import datetime

calvin['data_faturamento'] = pd.to_datetime(calvin['data_faturamento'], format='%d/%m/%Y')
calvin['data_lancamento'] = pd.to_datetime(calvin['data_lancamento'], format='%d/%m/%Y')
calvin['data_pagamento'] = pd.to_datetime(calvin['data_pagamento'], format='%d/%m/%Y')

In [9]:
calvin['forma_pgto'] = calvin['forma_pgto'].str.extract(r'^(\d+(?:/\d+)*)')[0]

In [10]:
calvin.nome_arquivo.unique()

array(['abril 23.csv', 'ago 23.csv', 'dez 23.csv', 'fev 23.csv',
       'jul 23.csv', 'jun 23.csv', 'maio 23.csv', 'marco 23.csv',
       'nov 23.csv', 'out 23.csv', 'set 23.csv'], dtype=object)

In [11]:
df = calvin

In [12]:
# Extrair o mês e o dia do nome do arquivo
df['mes'] = df['nome_arquivo'].str.split().str[0]

# Mapear o nome do mês para o número do mês
meses = {
    'jan': 1, 'fev': 2, 'marco': 3, 'abril': 4,
    'maio': 5, 'jun': 6, 'jul': 7, 'ago': 8,
    'set': 9, 'out': 10, 'nov': 11, 'dez': 12
}
df['mes'] = df['mes'].map(meses)


In [13]:
#preenchemos a data de devolução conforme o mes do relatorio de comissao
condicao = (df['tipo'] == 'DEVOLUCAO')
df.loc[condicao, 'data_pagamento'] = pd.to_datetime(df.loc[condicao, 'mes'], format='%m').apply(lambda x: pd.Timestamp(year=2023, month=x.month, day=1))

In [14]:
abril = calvin.query("nome_arquivo == 'abril 23.csv' ")

In [15]:
abril[['tipo','valor','comissao']].groupby('tipo').sum()

,valor,comissao
tipo,,
DEVOLUCAO,-51769.64,-2536.72
FATURAMENTO,2542130.80,124564.38
PAGAMENTOS,1327198.43,65032.70


In [16]:
abril[['data_pagamento','valor','comissao']].groupby(pd.Grouper(key='data_pagamento', freq='M')).sum()

,valor,comissao
data_pagamento,,
2023-04-30,1275428.79,62495.98


Nota-se que a data de pagamento se refere aos pagamentos recebidos no mesmo mês que o arquivo do comissionamento, no entanto, para a Jump este valor irá entrar no mês seguinte.

Caso: O cliente pagou a calvin em abril, a calvin vai pagar a jump em maio

In [17]:
calvin.query("documento == 940997 ").sort_values(by='data_pagamento')

,nome_arquivo,cliente,cidade,uf,tipo,documento,forma_pgto,valor,imposto,encargo,comissao_base,comissao,adiantamento,data_faturamento,data_lancamento,data_pagamento,parcela,representante,indicador_fiscal_terceiro,descricao_fiscal_terceiro,mes
11680,maio 23.csv,AAC COMERCIO DE CONFECCOE,FLORIANOPOLIS,SC,PAGAMENTOS,940997,28/49/70/91/112,1246.01,0,24.92,1221.09,61.05,30.53,2023-04-04,2023-04-04,2023-05-03,A,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,5
11681,maio 23.csv,AAC COMERCIO DE CONFECCOE,FLORIANOPOLIS,SC,PAGAMENTOS,940997,28/49/70/91/112,1246.01,0,24.92,1221.09,61.05,30.53,2023-04-04,2023-04-04,2023-05-24,B,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,5
9392,jun 23.csv,AAC COMERCIO DE CONFECCOE,FLORIANOPOLIS,SC,PAGAMENTOS,940997,28/49/70/91/112,1246.01,0,24.92,1221.09,61.05,30.53,2023-04-04,2023-04-04,2023-06-14,C,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,6
7428,jul 23.csv,AAC COMERCIO DE CONFECCOE,FLORIANOPOLIS,SC,PAGAMENTOS,940997,28/49/70/91/112,1246.01,0,24.92,1221.09,61.05,30.53,2023-04-04,2023-04-04,2023-07-04,D,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,7
7429,jul 23.csv,AAC COMERCIO DE CONFECCOE,FLORIANOPOLIS,SC,PAGAMENTOS,940997,28/49/70/91/112,1246.01,0,24.92,1221.09,61.05,30.53,2023-04-04,2023-04-04,2023-07-24,E,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,7
0,abril 23.csv,AAC COMERCIO DE CONFECCOE,FLORIANOPOLIS,SC,FATURAMENTO,940997,28/49/70/91/112,6230.04,0,124.60,6105.44,305.27,152.64,2023-04-04,NaT,NaT,NaN,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,4


In [18]:
calvin_faturamento = calvin.query("tipo == 'FATURAMENTO' ")

In [19]:
calvin_pagamento = calvin.query("tipo == 'PAGAMENTOS' ")

In [20]:
# novas_linhas = []
# for index, row in calvin_faturamento.iterrows():
#     if isinstance(row['forma_pgto'], str):
#         for idx, i in enumerate(row['forma_pgto'].split('/'), start=1):
#                 cliente = row['cliente'].str.lower()
#                 documento = row['documento']
#                 valor = row['valor']/len(row.forma_pgto.split('/'))
#                 encargo = row['encargo']/len(row.forma_pgto.split('/'))
#                 comissao_base = row['comissao_base']/len(row.forma_pgto.split('/'))
#                 comissao = row['comissao']/len(row.forma_pgto.split('/'))
#                 adiantamento = row['adiantamento']/len(row.forma_pgto.split('/'))
#                 parcela = idx
#                 data_faturamento = row['data_faturamento']
#                 data_pagamento = pd.Timestamp(row['data_faturamento']) + pd.Timedelta(days=int(i))
#                 novas_linhas.append({'documento':documento,
#                                     'valor':valor,
#                                     'encargo':encargo,
#                                     'comissao_base':comissao_base,
#                                     'comissao':comissao,
#                                     'adiantamento':adiantamento,
#                                     'parcela':parcela,
#                                     'data_faturamento': data_faturamento,
#                                     'data_pagamento': data_pagamento})
#     else:
#         cliente = row['cliente'].str.lower()
#         documento = row['documento']
#         valor = row['valor']
#         encargo = row['encargo']
#         comissao_base = row['comissao_base']
#         comissao = row['comissao']
#         adiantamento = row['adiantamento']
#         parcela = idx
#         data_faturamento = row['data_faturamento']
#         data_pagamento = pd.Timestamp(row['data_faturamento']) + pd.Timedelta(days=int(i))
#         novas_linhas.append({'documento':documento,
#                             'valor':valor,
#                             'encargo':encargo,
#                             'comissao_base':comissao_base,
#                             'comissao':comissao,
#                             'adiantamento':adiantamento,
#                             'parcela':parcela,
#                             'data_faturamento': data_faturamento,
#                             'data_pagamento': data_pagamento})
#     previsao = pd.DataFrame(novas_linhas)


In [22]:
novas_linhas = []
for index, row in calvin_faturamento.iterrows():
    if isinstance(row['forma_pgto'], str):
        for idx, i in enumerate(row['forma_pgto'].split('/'), start=1):
                cliente = row['cliente'].lower()
                documento = row['documento']
                valor = round(row['valor']/len(row.forma_pgto.split('/')),2)
                comissao = round(row['comissao']/len(row.forma_pgto.split('/')),2)
                parcela = idx
                data_faturamento = row['data_faturamento']
                data_pagamento = pd.Timestamp(row['data_faturamento']) + pd.Timedelta(days=int(i))
                novas_linhas.append({
                                    'cliente':cliente,
                                    'tipo':'previsao',
                                    'documento':documento,
                                    'valor':valor,
                                    'comissao':comissao,
                                    'parcela':parcela,
                                    'data_faturamento': data_faturamento,
                                    'data_pagamento': data_pagamento})
    else:
        cliente = row['cliente'].lower()
        documento = row['documento']
        valor = row['valor']
        comissao = row['comissao']
        parcela = idx
        data_faturamento = row['data_faturamento']
        data_pagamento = pd.Timestamp(row['data_faturamento']) + pd.Timedelta(days=int(i))
        novas_linhas.append({
                            'cliente':cliente,
                            'tipo':'previsao',
                            'documento':documento,
                            'valor':valor,
                            'comissao':comissao,
                            'parcela':parcela,
                            'data_faturamento': data_faturamento,
                            'data_pagamento': data_pagamento})
    previsao = pd.DataFrame(novas_linhas)


A previsão será feita a partir do relatório de comissao, tipo FATURAMENTO, onde iremos somar o número de dias da coluna forma de pagamento a data de faturamento, obtendo assim a previsão da receita

In [23]:
# Mapeamento de valores
mapa = {1: 'A', 2: 'B', 3: 'C', 4: 'D', 5: 'E'}

# Substituindo os valores
previsao['parcela'] = previsao['parcela'].replace(mapa)

In [24]:
previsao

,cliente,tipo,documento,valor,comissao,parcela,data_faturamento,data_pagamento
0,aac comercio de confeccoe,previsao,940997,1246.01,61.05,A,2023-04-04,2023-05-02
1,aac comercio de confeccoe,previsao,940997,1246.01,61.05,B,2023-04-04,2023-05-23
2,aac comercio de confeccoe,previsao,940997,1246.01,61.05,C,2023-04-04,2023-06-13
3,aac comercio de confeccoe,previsao,940997,1246.01,61.05,D,2023-04-04,2023-07-04
4,aac comercio de confeccoe,previsao,940997,1246.01,61.05,E,2023-04-04,2023-07-25
...,...,...,...,...,...,...,...,...
18054,zanoello & zanoello,previsao,980097,1167.09,57.19,A,2023-09-28,2023-10-26
18055,zanoello & zanoello,previsao,980097,1167.09,57.19,B,2023-09-28,2023-11-16
18056,zanoello & zanoello,previsao,980097,1167.09,57.19,C,2023-09-28,2023-12-07
18057,zanoello & zanoello,previsao,980097,1167.09,57.19,D,2023-09-28,2023-12-28


In [25]:
calvin.query("documento == 940997 ").sort_values(by='data_pagamento')

,nome_arquivo,cliente,cidade,uf,tipo,documento,forma_pgto,valor,imposto,encargo,comissao_base,comissao,adiantamento,data_faturamento,data_lancamento,data_pagamento,parcela,representante,indicador_fiscal_terceiro,descricao_fiscal_terceiro,mes
11680,maio 23.csv,AAC COMERCIO DE CONFECCOE,FLORIANOPOLIS,SC,PAGAMENTOS,940997,28/49/70/91/112,1246.01,0,24.92,1221.09,61.05,30.53,2023-04-04,2023-04-04,2023-05-03,A,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,5
11681,maio 23.csv,AAC COMERCIO DE CONFECCOE,FLORIANOPOLIS,SC,PAGAMENTOS,940997,28/49/70/91/112,1246.01,0,24.92,1221.09,61.05,30.53,2023-04-04,2023-04-04,2023-05-24,B,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,5
9392,jun 23.csv,AAC COMERCIO DE CONFECCOE,FLORIANOPOLIS,SC,PAGAMENTOS,940997,28/49/70/91/112,1246.01,0,24.92,1221.09,61.05,30.53,2023-04-04,2023-04-04,2023-06-14,C,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,6
7428,jul 23.csv,AAC COMERCIO DE CONFECCOE,FLORIANOPOLIS,SC,PAGAMENTOS,940997,28/49/70/91/112,1246.01,0,24.92,1221.09,61.05,30.53,2023-04-04,2023-04-04,2023-07-04,D,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,7
7429,jul 23.csv,AAC COMERCIO DE CONFECCOE,FLORIANOPOLIS,SC,PAGAMENTOS,940997,28/49/70/91/112,1246.01,0,24.92,1221.09,61.05,30.53,2023-04-04,2023-04-04,2023-07-24,E,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,7
0,abril 23.csv,AAC COMERCIO DE CONFECCOE,FLORIANOPOLIS,SC,FATURAMENTO,940997,28/49/70/91/112,6230.04,0,124.60,6105.44,305.27,152.64,2023-04-04,NaT,NaT,NaN,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,4


In [26]:
previsao.query("documento == 940997 ")

,cliente,tipo,documento,valor,comissao,parcela,data_faturamento,data_pagamento
0,aac comercio de confeccoe,previsao,940997,1246.01,61.05,A,2023-04-04,2023-05-02
1,aac comercio de confeccoe,previsao,940997,1246.01,61.05,B,2023-04-04,2023-05-23
2,aac comercio de confeccoe,previsao,940997,1246.01,61.05,C,2023-04-04,2023-06-13
3,aac comercio de confeccoe,previsao,940997,1246.01,61.05,D,2023-04-04,2023-07-04
4,aac comercio de confeccoe,previsao,940997,1246.01,61.05,E,2023-04-04,2023-07-25


In [27]:
calvin.query("documento == 928814 ").sort_values(by='data_pagamento')

,nome_arquivo,cliente,cidade,uf,tipo,documento,forma_pgto,valor,imposto,encargo,comissao_base,comissao,adiantamento,data_faturamento,data_lancamento,data_pagamento,parcela,representante,indicador_fiscal_terceiro,descricao_fiscal_terceiro,mes
11917,maio 23.csv,BEKKO BLUMENAU,BLUMENAU,SC,PAGAMENTOS,928814,28/49/70/91/112,1087.17,0,21.74,1065.43,53.27,26.64,2023-02-11,2023-02-11,2023-05-25,A,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,5
11918,maio 23.csv,BEKKO BLUMENAU,BLUMENAU,SC,PAGAMENTOS,928814,28/49/70/91/112,1087.17,0,21.74,1065.43,53.27,26.64,2023-02-11,2023-02-11,2023-05-25,B,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,5
11919,maio 23.csv,BEKKO BLUMENAU,BLUMENAU,SC,PAGAMENTOS,928814,28/49/70/91/112,1087.17,0,21.74,1065.43,53.27,26.64,2023-02-11,2023-02-11,2023-05-25,C,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,5
11920,maio 23.csv,BEKKO BLUMENAU,BLUMENAU,SC,PAGAMENTOS,928814,28/49/70/91/112,1087.17,0,21.74,1065.43,53.27,26.64,2023-02-11,2023-02-11,2023-05-25,D,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,5
11921,maio 23.csv,BEKKO BLUMENAU,BLUMENAU,SC,PAGAMENTOS,928814,28/49/70/91/112,1087.17,0,21.74,1065.43,53.27,26.64,2023-02-11,2023-02-11,2023-05-25,E,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,5
5985,fev 23.csv,BEKKO BLUMENAU,BLUMENAU,SC,FATURAMENTO,928814,28/49/70/91/112,5435.83,0,108.72,5327.11,266.36,133.18,2023-02-11,NaT,NaT,NaN,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,2


In [28]:
previsao.query("documento == 928814 ")

,cliente,tipo,documento,valor,comissao,parcela,data_faturamento,data_pagamento
4554,bekko blumenau,previsao,928814,1087.17,53.27,A,2023-02-11,2023-03-11
4555,bekko blumenau,previsao,928814,1087.17,53.27,B,2023-02-11,2023-04-01
4556,bekko blumenau,previsao,928814,1087.17,53.27,C,2023-02-11,2023-04-22
4557,bekko blumenau,previsao,928814,1087.17,53.27,D,2023-02-11,2023-05-13
4558,bekko blumenau,previsao,928814,1087.17,53.27,E,2023-02-11,2023-06-03


Nota, que pode ocorrer do cliente pagar os boletos tudo de uma vez, o que gerara divergencia entre a previsão e o real. Alem de que no real, teremos as devoluções também, o que não é considerado na previsão

In [29]:
previsao['mes_ano'] = previsao['data_pagamento'].dt.strftime('%m-%Y')
calvin['mes_ano'] = calvin['data_pagamento'].dt.strftime('%m-%Y')

In [30]:
previsao_agrupado = previsao[['mes_ano','comissao']].groupby('mes_ano').sum()
calvin_agrupado = calvin[['mes_ano','comissao']].groupby('mes_ano').sum()

In [31]:
pd.merge(previsao_agrupado,calvin_agrupado,on='mes_ano')

,comissao_x,comissao_y
mes_ano,,
03-2023,23916.31,33681.16
04-2023,68973.42,62495.98
05-2023,116425.39,113212.29
06-2023,134188.75,120234.10
07-2023,111025.62,105309.29
08-2023,99569.58,101776.23
09-2023,65191.55,62673.68
10-2023,103975.54,96083.80
11-2023,123099.89,112757.83


## TABELA REAL

In [32]:
historico = calvin.query("tipo != 'FATURAMENTO'")

In [33]:
historico.columns

Index(['nome_arquivo', 'cliente', 'cidade', 'uf', 'tipo', 'documento',
       'forma_pgto', 'valor', 'imposto', 'encargo', 'comissao_base',
       'comissao', 'adiantamento', 'data_faturamento', 'data_lancamento',
       'data_pagamento', 'parcela', 'representante',
       'indicador_fiscal_terceiro', 'descricao_fiscal_terceiro', 'mes',
       'mes_ano'],
      dtype='object')

In [34]:
historico = historico[['cliente', 'tipo', 'documento','valor','comissao','parcela', 'data_faturamento','data_pagamento']]

In [35]:
historico.cliente = historico.cliente.str.lower()
historico.tipo = historico.tipo.str.lower()


In [36]:
historico

,cliente,tipo,documento,valor,comissao,parcela,data_faturamento,data_pagamento
1,aac comercio de confeccoe,pagamentos,928624,1098.08,53.81,B,2023-02-10,2023-04-06
2,aac comercio de confeccoe,pagamentos,928624,688.99,33.76,C,2023-02-10,2023-04-26
3,aac comercio de confeccoe,pagamentos,936605,1284.36,62.93,A,2023-03-15,2023-04-12
8,adriana mariani,pagamentos,930162,1037.89,50.86,B,2023-02-17,2023-04-12
9,adriana mariani,pagamentos,930162,1037.89,50.86,C,2023-02-17,2023-04-28
...,...,...,...,...,...,...,...,...
21685,zaine tecidos,pagamentos,965191,1178.89,57.77,B,2023-07-25,2023-09-13
21686,zaine tecidos,pagamentos,965762,1377.27,67.49,B,2023-07-31,2023-09-19
21691,zanoello & zanoello,pagamentos,949735,1044.99,51.20,D,2023-05-12,2023-09-04
21692,zanoello & zanoello,pagamentos,966177,1040.53,50.99,B,2023-08-02,2023-09-20


In [37]:
historico[['valor','comissao','documento']].query("documento == 928624" ).sum()

valor           5490.39
comissao         269.05
documento    6500368.00
dtype: float64

In [38]:
calvin.query("documento == 928624 and tipo == 'FATURAMENTO' ")

,nome_arquivo,cliente,cidade,uf,tipo,documento,forma_pgto,valor,imposto,encargo,comissao_base,comissao,adiantamento,data_faturamento,data_lancamento,data_pagamento,parcela,representante,indicador_fiscal_terceiro,descricao_fiscal_terceiro,mes,mes_ano
5831,fev 23.csv,AAC COMERCIO DE CONFECCOE,FLORIANOPOLIS,SC,FATURAMENTO,928624,28/49/70/91/112,5490.38,0,109.81,5380.57,269.03,134.51,2023-02-10,NaT,NaT,NaN,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,2,NaN


In [39]:
calvin.query("nome_arquivo == 'fev 23.csv' and tipo == 'FATURAMENTO' ")

,nome_arquivo,cliente,cidade,uf,tipo,documento,forma_pgto,valor,imposto,encargo,comissao_base,comissao,adiantamento,data_faturamento,data_lancamento,data_pagamento,parcela,representante,indicador_fiscal_terceiro,descricao_fiscal_terceiro,mes,mes_ano
5831,fev 23.csv,AAC COMERCIO DE CONFECCOE,FLORIANOPOLIS,SC,FATURAMENTO,928624,28/49/70/91/112,5490.38,0,109.81,5380.57,269.03,134.51,2023-02-10,NaT,NaT,NaN,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,2,NaN
5833,fev 23.csv,ADRIANA MARIANI,ALFREDO WAGNER,SC,FATURAMENTO,930162,28/49/70/91/112,5189.46,0,103.79,5085.67,254.28,127.14,2023-02-17,NaT,NaT,NaN,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,2,NaN
5841,fev 23.csv,ADRIANO MANES,GUARAMIRIM,SC,FATURAMENTO,928225,28/49/70/91/112,5459.94,0,109.20,5350.74,267.54,133.77,2023-02-09,NaT,NaT,NaN,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,2,NaN
5842,fev 23.csv,ADRIANO MANES,GUARAMIRIM,SC,FATURAMENTO,931936,28/49/70/91/112,5254.52,0,105.09,5149.43,257.47,128.74,2023-02-24,NaT,NaT,NaN,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,2,NaN
5854,fev 23.csv,ALAIR DE SA PEREIRA,PALHOCA,SC,FATURAMENTO,928220,28/49/70/91/112,5493.49,0,109.87,5383.62,269.18,134.59,2023-02-09,NaT,NaT,NaN,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7391,fev 23.csv,VIVAR COMERCIO,XAXIM,SC,FATURAMENTO,929474,28/49/70/91/112,5387.22,0,107.74,5279.48,263.97,131.99,2023-02-15,NaT,NaT,NaN,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,2,NaN
7392,fev 23.csv,VIVAR COMERCIO,XAXIM,SC,FATURAMENTO,930928,28/49/70/91/112,5480.44,0,109.61,5370.83,268.54,134.27,2023-02-21,NaT,NaT,NaN,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,2,NaN
7394,fev 23.csv,W G MODA LTDA,TAIO,SC,FATURAMENTO,929499,28/49/70/91/112,5070.92,0,101.42,4969.50,248.48,124.24,2023-02-15,NaT,NaT,NaN,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,2,NaN
7395,fev 23.csv,W G MODA LTDA,TAIO,SC,FATURAMENTO,931965,28/49/70/91/112,5473.60,0,109.47,5364.13,268.21,134.10,2023-02-24,NaT,NaT,NaN,JUMP COMERCIO E REP,6,SIMPLES NACIONAL,2,NaN


## PREVISAO

In [40]:
previsao.query("documento == 928624" )

,cliente,tipo,documento,valor,comissao,parcela,data_faturamento,data_pagamento,mes_ano
4399,aac comercio de confeccoe,previsao,928624,1098.08,53.81,A,2023-02-10,2023-03-10,03-2023
4400,aac comercio de confeccoe,previsao,928624,1098.08,53.81,B,2023-02-10,2023-03-31,03-2023
4401,aac comercio de confeccoe,previsao,928624,1098.08,53.81,C,2023-02-10,2023-04-21,04-2023
4402,aac comercio de confeccoe,previsao,928624,1098.08,53.81,D,2023-02-10,2023-05-12,05-2023
4403,aac comercio de confeccoe,previsao,928624,1098.08,53.81,E,2023-02-10,2023-06-02,06-2023


# tabela final

In [60]:
df = calvin[['cliente', 'tipo', 'documento','valor','comissao','parcela', 'data_faturamento','data_pagamento']]
prev = previsao[['cliente', 'tipo', 'documento','valor','comissao','parcela', 'data_faturamento','data_pagamento']]
df_final = pd.concat([df,prev])
df_final['marca'] = 'calvin klein'
df_final.cliente = df_final.cliente.str.lower()
df_final.tipo = df_final.tipo.str.lower()
df_final.data_pagamento = df_final.data_pagamento.fillna(datetime(1900, 1, 1))

In [74]:
df_final

,cliente,tipo,documento,valor,comissao,parcela,data_faturamento,data_pagamento,marca
0,aac comercio de confeccoe,faturamento,940997,6230.04,305.27,NaN,2023-04-04,1900-01-01,calvin klein
1,aac comercio de confeccoe,pagamentos,928624,1098.08,53.81,B,2023-02-10,2023-04-06,calvin klein
2,aac comercio de confeccoe,pagamentos,928624,688.99,33.76,C,2023-02-10,2023-04-26,calvin klein
3,aac comercio de confeccoe,pagamentos,936605,1284.36,62.93,A,2023-03-15,2023-04-12,calvin klein
4,adriana mariani,faturamento,940891,7244.00,354.96,NaN,2023-04-04,1900-01-01,calvin klein
...,...,...,...,...,...,...,...,...,...
18054,zanoello & zanoello,previsao,980097,1167.09,57.19,A,2023-09-28,2023-10-26,calvin klein
18055,zanoello & zanoello,previsao,980097,1167.09,57.19,B,2023-09-28,2023-11-16,calvin klein
18056,zanoello & zanoello,previsao,980097,1167.09,57.19,C,2023-09-28,2023-12-07,calvin klein
18057,zanoello & zanoello,previsao,980097,1167.09,57.19,D,2023-09-28,2023-12-28,calvin klein


In [75]:
df_final.to_csv('calvin_consolidado.csv',index=False)

In [81]:
df_final[(df_final['data_faturamento'].dt.month == 11) & (df_final['data_faturamento'].dt.year == 2023) & (df_final['tipo'] == 'previsao')]

,cliente,tipo,documento,valor,comissao,parcela,data_faturamento,data_pagamento,marca,perc
12760,aac comercio de confeccoe,previsao,988519,1149.57,56.33,A,2023-11-13,2023-12-25,calvin klein,4.900222
12761,aac comercio de confeccoe,previsao,988519,1149.57,56.33,B,2023-11-13,2024-01-08,calvin klein,4.900222
12762,aac comercio de confeccoe,previsao,988519,1149.57,56.33,C,2023-11-13,2024-01-22,calvin klein,4.900222
12763,adriana mariani,previsao,986463,1011.56,49.57,A,2023-11-01,2023-11-29,calvin klein,4.900222
12764,adriana mariani,previsao,986463,1011.56,49.57,B,2023-11-01,2023-12-20,calvin klein,4.900222
...,...,...,...,...,...,...,...,...,...,...
14150,zanoello & zanoello,previsao,990031,1305.42,63.97,A,2023-11-20,2024-01-08,calvin klein,4.900222
14151,zanoello & zanoello,previsao,990031,1305.42,63.97,B,2023-11-20,2024-02-19,calvin klein,4.900222
14152,zanoello & zanoello,previsao,992172,489.54,23.99,A,2023-11-25,2024-02-03,calvin klein,4.900222
14153,zanoello & zanoello,previsao,992843,740.88,36.30,A,2023-11-27,2024-02-05,calvin klein,4.900222


In [89]:
df_final.query("tipo == 'pagamentos' and data_faturamento.dt.month == 11 and data_pagamento.dt.month == 11 ")

,cliente,tipo,documento,valor,comissao,parcela,data_faturamento,data_pagamento,marca,perc
15660,adriana mariani,pagamentos,986463,1011.56,49.57,A,2023-11-01,2023-11-30,calvin klein,4.900222
16250,contra tempo modas ltda,pagamentos,986377,1373.97,67.32,A,2023-11-01,2023-11-30,calvin klein,4.900222
16278,denig comercio de confecc,pagamentos,986652,1063.10,52.09,A,2023-11-03,2023-11-28,calvin klein,4.900222
16638,esportivos massocco,pagamentos,986428,1145.34,56.12,A,2023-11-01,2023-11-30,calvin klein,4.900222
17220,la belle boutique eireli,pagamentos,986393,1346.90,66.00,A,2023-11-01,2023-11-30,calvin klein,4.900222
17265,leli comercio,pagamentos,986433,1162.09,56.94,A,2023-11-01,2023-11-30,calvin klein,4.900222
17312,lena modas comercio de co,pagamentos,986335,1384.17,67.82,A,2023-11-01,2023-11-30,calvin klein,4.900222
17414,luciane dell antonia epp,pagamentos,986435,1050.81,51.49,A,2023-11-01,2023-11-30,calvin klein,4.900222
17468,madri comercio de artigos,pagamentos,986440,1116.67,54.72,A,2023-11-01,2023-11-30,calvin klein,4.900222
17480,marcel alexandre,pagamentos,986755,5328.04,261.07,A,2023-11-06,2023-11-28,calvin klein,4.900222


In [92]:
df_final.query("tipo == 'previsao' and data_faturamento.dt.month == 11 and data_pagamento.dt.month == 12 ")

,cliente,tipo,documento,valor,comissao,parcela,data_faturamento,data_pagamento,marca,perc
12760,aac comercio de confeccoe,previsao,988519,1149.57,56.33,A,2023-11-13,2023-12-25,calvin klein,4.900222
12764,adriana mariani,previsao,986463,1011.56,49.57,B,2023-11-01,2023-12-20,calvin klein,4.900222
12773,adriano manes,previsao,988520,1000.44,49.02,A,2023-11-13,2023-12-11,calvin klein,4.900222
12781,alair de sa pereira,previsao,988070,1060.33,51.95,A,2023-11-10,2023-12-29,calvin klein,4.900222
12788,alarice comercio,previsao,987499,891.35,43.68,A,2023-11-08,2023-12-06,calvin klein,4.900222
...,...,...,...,...,...,...,...,...,...,...
14127,zaine tecidos,previsao,986686,1031.19,50.53,B,2023-11-06,2023-12-25,calvin klein,4.900222
14131,zaine tecidos,previsao,988206,1173.77,57.52,A,2023-11-10,2023-12-29,calvin klein,4.900222
14141,zanoello & zanoello,previsao,986341,1210.64,59.32,B,2023-11-01,2023-12-20,calvin klein,4.900222
14145,zanoello & zanoello,previsao,988361,1397.09,68.46,A,2023-11-10,2023-12-08,calvin klein,4.900222


In [99]:
df_final[(df_final['cliente'].str.contains('julia grando')) & (df_final.data_pagamento.dt.month == 6) & (df_final.tipo == 'pagamentos')]

,cliente,tipo,documento,valor,comissao,parcela,data_faturamento,data_pagamento,marca,perc
10686,julia grando,pagamentos,940237,2186.04,107.12,A,2023-03-31,2023-06-09,calvin klein,4.900222
10687,julia grando,pagamentos,940239,3267.11,160.09,A,2023-03-31,2023-06-09,calvin klein,4.900222
10688,julia grando,pagamentos,940242,2269.44,111.20,A,2023-03-31,2023-06-07,calvin klein,4.900222
10689,julia grando,pagamentos,940289,2153.37,105.52,A,2023-03-31,2023-06-09,calvin klein,4.900222
10690,julia grando,pagamentos,940330,2216.65,108.62,A,2023-03-31,2023-06-09,calvin klein,4.900222
10691,julia grando,pagamentos,942453,1999.44,97.97,A,2023-04-11,2023-06-22,calvin klein,4.900222
10692,julia grando,pagamentos,943234,1236.72,60.60,A,2023-04-13,2023-06-27,calvin klein,4.900222
10693,julia grando,pagamentos,943515,1749.00,85.70,A,2023-04-14,2023-06-27,calvin klein,4.900222
10694,julia grando,pagamentos,944112,753.03,36.90,D,2023-04-18,2023-06-19,calvin klein,4.900222


In [103]:
df_final[(df_final['cliente'].str.contains('julia grando')) & (df_final.tipo == 'faturamento')]

,cliente,tipo,documento,valor,comissao,parcela,data_faturamento,data_pagamento,marca,perc
824,julia grando,faturamento,942453,9997.20,489.86,NaN,2023-04-11,1900-01-01,calvin klein,4.900222
825,julia grando,faturamento,943234,6183.62,303.00,NaN,2023-04-13,1900-01-01,calvin klein,4.900222
826,julia grando,faturamento,943515,8745.01,428.51,NaN,2023-04-14,1900-01-01,calvin klein,4.900222
827,julia grando,faturamento,944112,8118.73,397.82,NaN,2023-04-18,1900-01-01,calvin klein,4.900222
13192,julia grando,faturamento,948919,6565.09,321.69,NaN,2023-05-10,1900-01-01,calvin klein,4.900222
13193,julia grando,faturamento,948970,7991.40,391.58,NaN,2023-05-10,1900-01-01,calvin klein,4.900222
15120,julia grando,faturamento,940237,10930.18,535.58,NaN,2023-03-31,1900-01-01,calvin klein,4.900222
15121,julia grando,faturamento,940239,16335.56,800.44,NaN,2023-03-31,1900-01-01,calvin klein,4.900222
15122,julia grando,faturamento,940242,11347.22,556.01,NaN,2023-03-31,1900-01-01,calvin klein,4.900222
15123,julia grando,faturamento,940243,10449.67,512.03,NaN,2023-03-31,1900-01-01,calvin klein,4.900222


In [100]:
df_final[(df_final['cliente'].str.contains('julia grando')) & (df_final.data_pagamento.dt.month == 6) & (df_final.tipo == 'previsao')]

,cliente,tipo,documento,valor,comissao,parcela,data_faturamento,data_pagamento,marca,perc
1011,julia grando,previsao,942453,1999.44,97.97,C,2023-04-11,2023-06-20,calvin klein,4.900222
1015,julia grando,previsao,943234,1236.72,60.60,B,2023-04-13,2023-06-01,calvin klein,4.900222
1016,julia grando,previsao,943234,1236.72,60.60,C,2023-04-13,2023-06-22,calvin klein,4.900222
1020,julia grando,previsao,943515,1749.00,85.70,B,2023-04-14,2023-06-02,calvin klein,4.900222
1021,julia grando,previsao,943515,1749.00,85.70,C,2023-04-14,2023-06-23,calvin klein,4.900222
1025,julia grando,previsao,944112,1623.75,79.56,B,2023-04-18,2023-06-06,calvin klein,4.900222
1026,julia grando,previsao,944112,1623.75,79.56,C,2023-04-18,2023-06-27,calvin klein,4.900222
9066,julia grando,previsao,948919,1313.02,64.34,A,2023-05-10,2023-06-07,calvin klein,4.900222
9067,julia grando,previsao,948919,1313.02,64.34,B,2023-05-10,2023-06-28,calvin klein,4.900222
9071,julia grando,previsao,948970,1598.28,78.32,A,2023-05-10,2023-06-07,calvin klein,4.900222


In [93]:
df_final.data_pagamento.dt.year.unique()

array([1900, 2023, 2024])